In [8]:
# 1) Install packages in your environment (uncomment if running in Colab or local script):
# install dependencies
!pip install -r requirements.txt

# load env variables


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os
import sqlite3
import logging

from langchain import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

In [10]:
# -----------------------------------------------
# 2) Create a Small SQLite Database (In-Memory)
# -----------------------------------------------
# Option A: In-memory DB via SQLAlchemy URI
#   The "sqlite:///:memory:" URI doesn't work out-of-the-box with direct table creation 
#   using SQLDatabase.from_uri, so we'll do a hybrid approach:
#   - Manually create the table with sqlite3
#   - Then re-connect via LangChain’s SQLDatabase

db_uri = "file:langchain_temp_db?mode=memory&cache=shared"
full_uri = f"sqlite:///{db_uri}"

# Create the in-memory database using sqlite3
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create a toy "employees" table
cursor.execute("""
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        country TEXT,
        salary INTEGER
    )
""")

# Insert sample rows
employees_data = [
    (1, 'Alice',   'USA',     100000),
    (2, 'Bob',     'Germany', 90000),
    (3, 'Charlie', 'USA',     120000),
    (4, 'Diana',   'France',  95000),
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees_data)
conn.commit()


In [11]:
# We'll keep this connection open, but let's wrap it with LangChain's SQLDatabase
# by attaching to the *same* in-memory DB through a "shared cache" trick.
# (SQLAlchemy can’t directly share the same connection object, so we do the following).
#   1) Dump data to a temporary on-disk db or
#   2) Use the special URI syntax for shared in-memory

# For a truly ephemeral approach, let's switch to a file-based approach quickly:
conn.close()
temp_db_file = "temp_langchain_employees.db"
# Recreate on disk so we can read it with SQLDatabase
conn_disk = sqlite3.connect(temp_db_file)
cur_disk = conn_disk.cursor()
cur_disk.execute("""
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        country TEXT,
        salary INTEGER
    )
""")
cur_disk.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", employees_data)
conn_disk.commit()
conn_disk.close()

# Now we can use that file-based SQLite DB with LangChain’s SQLDatabase
db = SQLDatabase.from_uri(f"sqlite:///{temp_db_file}")


OperationalError: table employees already exists

In [12]:
# create an agent:


# 2) Create a new OpenAI instance
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

# 3) Create a new SQLDatabase instance
db = SQLDatabase.from_uri("sqlite:///chinook.db")

# 4) Create a new SQLAgent instance
agent = create_sql_agent(llm=llm, db=db, verbose=True )

# 5) Run the agent
agent.run("How many employees are there?")



> Entering new SQL Agent Executor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}